In [2]:
import my_sdk
import pandas as pd
from sklearn.datasets import load_iris
from deltalake.writer import write_deltalake
import pyarrow as pa

my_sdk.init()
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df['features_list'] = df[data.feature_names].values.tolist()
df = df[['features_list', 'target']]

print(df.head())

table_path = "s3://mybucket/iris-delta/"
writer = my_sdk.initiate_data_writer(table_path, mode="overwrite")

pa_table = pa.Table.from_pandas(df)
my_sdk.write_data(writer, pa_table)
print("Data preparation complete!")

--- [MLOps Platform SDK] ---
Ray is already connected.
Backend API configured at: http://localhost:8000
--- SDK Ready ---
          features_list  target
0  [5.1, 3.5, 1.4, 0.2]       0
1  [4.9, 3.0, 1.4, 0.2]       0
2  [4.7, 3.2, 1.3, 0.2]       0
3  [4.6, 3.1, 1.5, 0.2]       0
4  [5.0, 3.6, 1.4, 0.2]       0
Data preparation complete!


In [3]:
import numpy as np
import my_sdk
from sklearn.linear_model import SGDClassifier

class MySKLearnTrainer(my_sdk.BaseTrainWorker):

    def init_model(self):
        return SGDClassifier()

    def initialize(self):
        print("SGDClassifier model initialized.")

    def train_step(self, model, batch, epoch, config):
        features_batch = np.stack(batch['features_list'])        
        labels_batch = np.array(batch['target'])
        model.partial_fit(features_batch, labels_batch, classes=np.array([0, 1, 2]))
        
        return {"epoch": epoch}

    def post_epoch(self, epoch, metrics):
        print(f" Finished Epoch {epoch}. Metrics count: {len(metrics)}")

    def post_train(self, final_metrics):
        print("Training is complete.")

In [4]:
config = my_sdk.TrainConfig(
    model_name="ModelTrainedWithNotebook",
    delta_path="s3://mybucket/iris-delta/",
    epochs=50,
    batch_size=32,
    num_workers=1
)

print(f"Starting training for model: {config.model_name}")
result = my_sdk.train(MySKLearnTrainer, config)

Starting training for model: ModelTrainedWithNotebook
[Driver] Starting Distributed Training on Ray Cluster
[Driver] Configuration:
  Model: ModelTrainedWithNotebook
  Workers: 1
  Epochs: 50
  Batch size: 32
  Shuffle seed: 42
  Custom run name: Auto-generated
----------------------------------------------------------------------
[Driver] Creating 1 Ray workers on cluster...
[Driver] Setting up workers (loading data & models on Ray cluster)...
(_TrainWorkerActor pid=764) [Worker 0] MLflow URI set to: http://mlflow-server:5000
(_TrainWorkerActor pid=764) [Worker 0] Starting setup on Ray cluster...
(_TrainWorkerActor pid=764) [Worker 0] Loading data from: s3://mybucket/iris-delta/


(_TrainWorkerActor pid=764) Estimated parquet encoding ratio is 3.629.
(_TrainWorkerActor pid=764) Estimated parquet reader batch size at 2982617 rows


(_TrainWorkerActor pid=764) [Worker 0] Initializing model...
(_TrainWorkerActor pid=764) SGDClassifier model initialized.
(_TrainWorkerActor pid=764) [Worker 0] Starting MLflow run: train_ModelTrainedWithNotebook_worker_0_e20824e5
(_TrainWorkerActor pid=764) --- [SDK] Finding notebook path to set experiment...
(_TrainWorkerActor pid=764) --- [SDK] Could not find notebook path. Using default experiment: 'MYMLOpsProjectExperiment' ---
[Driver] ✅ All workers ready. Each worker has started its own MLflow run.
----------------------------------------------------------------------
[Driver] Starting training...
----------------------------------------------------------------------
[Driver] Epoch 1/50
(_TrainWorkerActor pid=764) --- [SDK] Starting run: train_ModelTrainedWithNotebook_worker_0_e20824e5 (ID: 0c6ea02d491c4c9b90066b9f21b9e2d8) ---
(_TrainWorkerActor pid=764) [Worker 0] Setup complete. Data and model remain on Ray cluster.


(_TrainWorkerActor pid=764) Registered dataset logger for dataset dataset_1_0
(_TrainWorkerActor pid=764) Starting execution of Dataset dataset_1_0. Full logs are in /tmp/ray/session_2025-12-29_02-59-28_364723_7/logs/ray-data
(_TrainWorkerActor pid=764) Execution plan of Dataset dataset_1_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> AllToAllOperator[RandomShuffle]
(_TrainWorkerActor pid=764) [dataset]: A new progress UI is available. To enable, set `ray.data.DataContext.get_current().enable_rich_progress_bars = True` and `ray.data.DataContext.get_current().use_ray_tqdm = False`.
(_TrainWorkerActor pid=764) ⚠️  Ray's object store is configured to use only 42.9% of available memory (8.9GiB out of 20.8GiB total). For optimal Ray Data performance, we recommend setting the object store to at least 50% of available memory. You can do this by setting the 'object_store_memory' parameter when calling ray.init() or by setting the RAY_DEFAULT_OBJECT_STORE_MEMORY_PROPORTION env

(_TrainWorkerActor pid=764) [Worker] Epoch=0 Batch=0 Metrics={'epoch': 0, 'batch': 0}
(_TrainWorkerActor pid=764)  Finished Epoch 0. Metrics count: 1
(_TrainWorkerActor pid=764) [Worker] Epoch=0 Batch=1 Metrics={'epoch': 0, 'batch': 1}
(_TrainWorkerActor pid=764)  Finished Epoch 0. Metrics count: 2
(_TrainWorkerActor pid=764) [Worker] Epoch=0 Batch=2 Metrics={'epoch': 0, 'batch': 2}
(_TrainWorkerActor pid=764)  Finished Epoch 0. Metrics count: 3
[Driver]   ✅ Epoch 1 complete. Total batches processed: 5
[Driver] Epoch 2/50
(_TrainWorkerActor pid=764) [Worker] Epoch=0 Batch=3 Metrics={'epoch': 0, 'batch': 3}
(_TrainWorkerActor pid=764)  Finished Epoch 0. Metrics count: 4
(_TrainWorkerActor pid=764) [Worker] Epoch=0 Batch=4 Metrics={'epoch': 0, 'batch': 4}
(_TrainWorkerActor pid=764)  Finished Epoch 0. Metrics count: 5


...

(_TrainWorkerActor pid=764) [Worker] Epoch=49 Batch=0 Metrics={'epoch': 49, 'batch': 0}
(_TrainWorkerActor pid=764)  Finished Epoch 49. Metrics count: 1
(_TrainWorkerActor pid=764) [Worker] Epoch=49 Batch=1 Metrics={'epoch': 49, 'batch': 1}
(_TrainWorkerActor pid=764)  Finished Epoch 49. Metrics count: 2
(_TrainWorkerActor pid=764) [Worker] Epoch=49 Batch=2 Metrics={'epoch': 49, 'batch': 2}
(_TrainWorkerActor pid=764)  Finished Epoch 49. Metrics count: 3


(_TrainWorkerActor pid=764) ✔️  Dataset dataset_99_0 execution finished in 0.39 seconds
(_TrainWorkerActor pid=764) Registered dataset logger for dataset dataset_100_0


(_TrainWorkerActor pid=764) [Worker] Epoch=49 Batch=3 Metrics={'epoch': 49, 'batch': 3}
(_TrainWorkerActor pid=764)  Finished Epoch 49. Metrics count: 4
(_TrainWorkerActor pid=764) [Worker] Epoch=49 Batch=4 Metrics={'epoch': 49, 'batch': 4}
(_TrainWorkerActor pid=764)  Finished Epoch 49. Metrics count: 5
(_TrainWorkerActor pid=764) [Worker 0] Finalizing training...
(_TrainWorkerActor pid=764) Training is complete.
(_TrainWorkerActor pid=764) [Worker 0] Registering model to MLflow...
(_TrainWorkerActor pid=764) Attempting to register model ModelTrainedWithNotebook_worker_0.
(_TrainWorkerActor pid=764) Detected scikit-learn model.


(_TrainWorkerActor pid=764) ✔️  Dataset dataset_100_0 execution finished in 0.39 seconds
(_TrainWorkerActor pid=764) Exiting prefetcher's background thread
(_TrainWorkerActor pid=764) 2025/12/29 03:03:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
(_TrainWorkerActor pid=764) Successfully registered model 'ModelTrainedWithNotebook_worker_0'.
(_TrainWorkerActor pid=764) 2025/12/29 03:03:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ModelTrainedWithNotebook_worker_0, version 1
(_TrainWorkerActor pid=764) Created version '1' of model 'ModelTrainedWithNotebook_worker_0'.


[Driver] 🎉 Training Finished Successfully!
[Driver] Summary:
  - Trained 1 models (one per worker)
  - Each model logged to separate MLflow run
  - Models are in MLflow (NOT in this notebook)
  - To use models, load them from MLflow by run name
(_TrainWorkerActor pid=764) [Worker 0] Model registered: ModelTrainedWithNotebook_worker_0
(_TrainWorkerActor pid=764) [Worker 0] Ending MLflow run...
(_TrainWorkerActor pid=764) --- [SDK] Manually ending run: train_ModelTrainedWithNotebook_worker_0_e20824e5 ---
(_TrainWorkerActor pid=764) 🏃 View run train_ModelTrainedWithNotebook_worker_0_e20824e5 at: http://mlflow-server:5000/#/experiments/1/runs/0c6ea02d491c4c9b90066b9f21b9e2d8
(_TrainWorkerActor pid=764) 🧪 View experiment at: http://mlflow-server:5000/#/experiments/1
(_TrainWorkerActor pid=764) [Worker 0] Training complete. Model stays on Ray cluster/MLflow.
(_TrainWorkerActor pid=764) [Worker 0] To use model, load it from MLflow using run name: train_ModelTrainedWithNotebook_worker_0_e20824

In [5]:
import my_sdk
import numpy as np
from sklearn.linear_model import SGDClassifier

class MySKLearnTrainer(my_sdk.BaseTrainWorker):
    def init_model(self):
        return SGDClassifier()

    def initialize(self):
        print("SGDClassifier model initialized.")

    def train_step(self, model, batch, epoch, config):
        features_batch = np.stack(batch['features_list'])        
        labels_batch = np.array(batch['target'])
        model.partial_fit(features_batch, labels_batch, classes=np.array([0, 1, 2]))
        return {"epoch": epoch}

    def post_epoch(self, epoch, metrics):
        print(f" Finished Epoch {epoch}. Metrics count: {len(metrics)}")

    def post_train(self, final_metrics):
        print("Training is complete.")

In [6]:
TRAINER_NAME = "IrisTrainer"

trainer_response = my_sdk.register_trainer(
    trainer_cls=MySKLearnTrainer,
    name=TRAINER_NAME,
    description="A standard SGDClassifier trainer for the Iris dataset.",
    created_by="mmdyeah"
)

[SDK] Serializing trainer class 'MySKLearnTrainer'...
[SDK] Attempting to capture source code...
[SDK] Source code captured from IPython history (747 characters)
[SDK] Serialization successful! Size: 1809 bytes
[SDK] Sending serialized trainer to backend...
✅ Trainer 'IrisTrainer' registered successfully!
   ID: 1
   Serialization method: dill (includes all imports automatically)
   Created at: 2025-12-29T11:04:07.974671+00:00


In [7]:
config = my_sdk.TrainConfig(
    model_name="ModelTrainedWithNotebook",
    delta_path="s3://mybucket/iris-delta/",
    epochs=100,
    batch_size=128,
    num_workers=1
)

In [8]:
CONFIG_NAME = "IrisTrainerConfig"

config_response = my_sdk.register_train_config(
    config=config,
    name=CONFIG_NAME,
    trainer_name=TRAINER_NAME, 
    description="Production configuration with 100 epochs.",
    created_by="mmdyeah"
)

✅ Training config 'IrisTrainerConfig' registered successfully!
   ID: 2
   Trainer: IrisTrainer
   Created at: 2025-12-29T11:05:13.960953+00:00


In [10]:
import my_sdk
import numpy as np
import pandas as pd
import pyarrow as pa
from typing import Dict, Any

my_sdk.init()

class IrisPredictionWorker(my_sdk.BasePredictWorker):

    def initialize(self):
        print("[Worker Setup] Inference model successfully loaded.")

    def predict_batch(self, pyfunc_model, batch, config) -> Dict[str, Any]:

        features = np.stack(batch['features_list'])         
        predictions = pyfunc_model.predict(features)
        return {
            "input_features": batch['features_list'],
            "predicted_class": predictions,
            "original_target": batch['target']
        }

    def post_predict(self, all_outputs):
        print("[Worker Finalize] Prediction write complete for this shard.")


--- [MLOps Platform SDK] ---
Ray is already connected.
Backend API configured at: http://localhost:8000
--- SDK Ready ---


In [11]:
trainer_response = my_sdk.register_inference_engine(
    engine_cls=IrisPredictionWorker,
    name="IrisInferenceCode",
    description="infernce code for iris Model.",
    created_by="mmdyeah"
)

[SDK] Serializing inference engine class 'IrisPredictionWorker'...
[SDK] Attempting to capture source code...
[SDK] Source code captured from IPython history (751 characters)
[SDK] Serialization successful! Size: 1403 bytes
[SDK] Sending serialized inference engine to backend...
✅ Inference engine 'IrisInferenceCode' registered successfully (ID: 1)
   Serialization method: dill (includes all imports automatically)


In [12]:
INPUT_PATH="s3://mybucket/iris-delta/"
OUTPUT_PATH="s3://mybucket/iris-inference/"
inference_config = my_sdk.PredictionConfig(
    model_name="ModelTrainedWithNotebook",
    input_delta_path=INPUT_PATH,
    output_delta_path=OUTPUT_PATH,
    batch_size=128,
    num_workers=1
)


In [13]:
register_inference_config = my_sdk.register_inference_config(
    config=inference_config ,
    name="iris_inference_config" ,
    model_name = "ModelTrainedWithNotebook" ,
    description= "this is the config of the iris inference job." ,
    created_by="mmdyeah"
)

✅ Inference config 'iris_inference_config' registered successfully (ID: 1)
